# Ejercicio 1: Clasificación con Scikit-learn y MLflow

En esta práctica, utilizarás un conjunto de datos de Scikit-learn (podeís usar el mismo que en el notebook de Intro MLFlow) para entrenar un modelo de clasificación.

Pasos a seguir: 

    Exploración de Datos: Analiza el conjunto de datos proporcionado para comprender su estructura y contenido.

    Preprocesamiento de Texto: Realiza tareas de preprocesamiento de texto, como tokenización y vectorización, para preparar los datos para el modelado.

    Entrenamiento del Modelo: Utiliza algoritmos de clasificación de Scikit-learn para entrenar un modelo con los datos preprocesados.

    Evaluación del Modelo: Evalúa el rendimiento del modelo utilizando métricas de evaluación estándar como precisión y recall.

    Registro de Métricas con MLflow: Utiliza MLflow para registrar métricas y hiperparámetros durante el entrenamiento, facilitando la gestión y comparación de experimentos.


Nota: Dado que no voy a poder tener acceso a vuestros logs de MLFlow añadirme las imagenes de la interfaz de MLFlow en el notebook

In [74]:
pip install mlflow # Instalamos mlflow

In [75]:
pip install pyngrok # Instalamos ngrok

In [76]:
# Importamos las librerías necesarias

import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

from pyngrok import ngrok
from google.colab import userdata

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.datasets import load_wine
# Elijo este dataset para probar con uno diferente al de clase


Exploración de datos

In [77]:
wine = load_wine()
data = wine.data
target = wine.target
feature_names = wine.feature_names
target_names = wine.target_names

df = pd.DataFrame(data, columns=feature_names)
df['target'] = target

print(df.head())
print(f"Número de muestras (longitud del dataset): {len(data)}")
print(f"Número de características (X o variables independientes): {len(feature_names)}")
print(f"Número de clases (y - Dependiente a predecir): {len(target_names)}")


   alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0    14.23        1.71  2.43               15.6      127.0           2.80   
1    13.20        1.78  2.14               11.2      100.0           2.65   
2    13.16        2.36  2.67               18.6      101.0           2.80   
3    14.37        1.95  2.50               16.8      113.0           3.85   
4    13.24        2.59  2.87               21.0      118.0           2.80   

   flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
0        3.06                  0.28             2.29             5.64  1.04   
1        2.76                  0.26             1.28             4.38  1.05   
2        3.24                  0.30             2.81             5.68  1.03   
3        3.49                  0.24             2.18             7.80  0.86   
4        2.69                  0.39             1.82             4.32  1.04   

   od280/od315_of_diluted_wines  proline  target  
0          

In [78]:
df.describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258,0.938202
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474,0.775035
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000,0.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000,0.000000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000,1.000000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000,2.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000,2.000000


In [79]:
df.columns

Index(['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium',
       'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue',
       'od280/od315_of_diluted_wines', 'proline', 'target'],
      dtype='object')

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
 13  targe

Obviamente, es un dataset que no tenemos que preprocesar en términos de NaN o de feature engineering y tampoco es el objetivo de la práctica, por lo que, aunque me habría gustado introducir un dataset en el que estoy trabajando, por motivos de tiempo he decidido quedarme con este dataframe y trabajar con él tal como está.

División del conjunto de datos y preprocesado

In [81]:
# Lo hacemos igual que en clase

train, test = train_test_split(df, test_size=0.2)

test_target = test['target']
test[['target']].to_csv('test-target', index=False)
del test['target']
test.to_csv('test.csv', index=False)

features = [x for x in list(train.columns) if x != 'target']
x_raw = train[features]
y_raw = train['target']

X_train, X_test, y_train, y_test = train_test_split(x_raw, y_raw,
                                                            test_size=.20,
                                                            random_state=123,
                                                            stratify=y_raw)


In [82]:
preprocessor = Pipeline(steps=[('scaler', StandardScaler())])


Entrenamiento del modelo, evaluación y registro en MLFlow

In [83]:
# Lo hago a través de una función para entrenar dos modelos diferentes: LR y KNN

def train_and_log_model(model, params, model_name, run_name):
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('model', model)])

    # GridSearchCV para ver los mejores hiperparámetros
    grid_search = GridSearchCV(clf, params, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # Evaluación
    best_params = grid_search.best_params_
    accuracy_train = grid_search.score(X_train, y_train)
    accuracy_test = grid_search.score(X_test, y_test)
    y_pred = grid_search.predict(X_test)
    report = classification_report(y_test, y_pred, target_names=target_names, output_dict=True)

    # Registro de métricas en MLflow
    with mlflow.start_run(run_name=run_name) as run:
        run_id = run.info.run_id
        model_uri = f'runs:/{run_id}/{model_name}'

        mlflow.log_params(best_params)
        mlflow.log_metric('accuracy_train', accuracy_train)
        mlflow.log_metric('accuracy_test', accuracy_test)
        mlflow.sklearn.log_model(grid_search.best_estimator_, model_name)

        model_details = mlflow.register_model(
            model_uri=model_uri,
            name=model_name
        )

        # Guardamos el informe de clasificación como un artifact
        report_df = pd.DataFrame(report).transpose()
        report_path = f"{model_name}_classification_report.csv"
        report_df.to_csv(report_path, index=True)
        mlflow.log_artifact(report_path)

    # Resultados
    print(f"Model: {model_name}")
    print(f"Best Params: {best_params}")
    print(f"Accuracy Train: {accuracy_train}")
    print(f"Accuracy Test: {accuracy_test}")
    print(f"Classification Report: \n {report_df}\n")

In [84]:
# Modelo 1: Regresión Logística
logistic_model = LogisticRegression(max_iter=1000)
logistic_params = {
    "model__C": [0.1, 1, 10],
    "model__solver": ["liblinear", "lbfgs"]
}

# Modelo 2: K-Nearest Neighbors
knn_model = KNeighborsClassifier()
knn_params = {
    "model__n_neighbors": [3, 5, 7],
    "model__weights": ["uniform", "distance"]
}

train_and_log_model(logistic_model, logistic_params, 'LogisticRegressionModel', 'Logistic_Regression_Run')

train_and_log_model(knn_model, knn_params, 'KNNModel', 'KNN_Run')

Registered model 'LogisticRegressionModel' already exists. Creating a new version of this model...
Created version '4' of model 'LogisticRegressionModel'.


Model: LogisticRegressionModel
Best Params: {'model__C': 1, 'model__solver': 'liblinear'}
Accuracy Train: 1.0
Accuracy Test: 1.0
Classification Report: 
               precision  recall  f1-score  support
class_0             1.0     1.0       1.0     10.0
class_1             1.0     1.0       1.0     12.0
class_2             1.0     1.0       1.0      7.0
accuracy            1.0     1.0       1.0      1.0
macro avg           1.0     1.0       1.0     29.0
weighted avg        1.0     1.0       1.0     29.0

Model: KNNModel
Best Params: {'model__n_neighbors': 3, 'model__weights': 'uniform'}
Accuracy Train: 0.9823008849557522
Accuracy Test: 0.8620689655172413
Classification Report: 
               precision    recall  f1-score    support
class_0        0.833333  1.000000  0.909091  10.000000
class_1        1.000000  0.666667  0.800000  12.000000
class_2        0.777778  1.000000  0.875000   7.000000
accuracy       0.862069  0.862069  0.862069   0.862069
macro avg      0.870370  0.888889  

Registered model 'KNNModel' already exists. Creating a new version of this model...
Created version '2' of model 'KNNModel'.


Los resultados hay que revisarlos, sobre todo revisar si hay overfitting (ya que en un modelo tenemos una precisión perfecta), ver si debemos ampliar la muestra, hacer validación cruzada o ajustar hiperparámetros, pero como no es el objetivo de esta práctica, voy a seguir avanzando.

Nos conectamos a ngrok: Lo estoy haciendo en colab, por lo que he decidido coger el token de "secrets" que nos ofrece Google.

In [85]:
# Obtengo el token desde secrets de Google Colab
ngrok_token = userdata.get('ngrok')

ngrok.set_auth_token(ngrok_token)
ngrok.kill()

public_url = ngrok.connect(5000)
print("ngrok URL:", public_url)

# Para cerrar la sesión
input("Presiona Enter para finalizar la sesión de ngrok...")
ngrok.disconnect(public_url)

ngrok URL: NgrokTunnel: "https://bf34-34-72-212-101.ngrok-free.app" -> "http://localhost:5000"
Presiona Enter para finalizar la sesión de ngrok...


Adjunto capturas de pantalla de MLFlow

![Modelos inicio colab](./images_mlflow/Modelos_inicio_colab.png).

![Modelos inicio colab](./images_mlflow/KNN_Overview_colab.png).

![Modelos inicio colab](./images_mlflow/KNN_Metrics_colab.png).

![Modelos inicio colab](./images_mlflow/KNN_Artifacts_colab.png).

![Modelos inicio colab](./images_mlflow/KNN_Report_colab.png).

![Modelos inicio colab](./images_mlflow/LR_Overview_colab.png).

![Modelos inicio colab](./images_mlflow/LR_Metrics_colab.png).

![Modelos inicio colab](./images_mlflow/LR_Artifacts_colab.png).

![Modelos inicio colab](./images_mlflow/LR_Report_colab.png).